In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '215_eda'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
feature:
    name_bow_pca_dim: 10
    name_bow_word_th1: 5
    name_bow_word_th2: 3
    name_bow_th1_upper: 130
    name_bow_th2_upper: 1000
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])
pd.set_option('display.max_rows', 500)

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


In [9]:
df_train.loc[:, 'has_na_sales'] = df_train.loc[:, 'NA_Sales']  > 0
df_train.loc[:, 'has_eu_sales'] = df_train.loc[:, 'EU_Sales']  > 0
df_train.loc[:, 'has_jp_sales'] = df_train.loc[:, 'JP_Sales']  > 0
df_train.loc[:, 'has_other_sales'] = df_train.loc[:, 'Other_Sales']  > 0
# df_train.to_csv(datadir / 'country_train.csv', index=False)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# 要検討
df_best_pred = pd.read_csv('../ckpt/187_name_count/187_name_count.csv')
df_test.loc[:, 'Global_Sales'] = df_best_pred.loc[:, 'Global_Sales']

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
train = df_train.copy()
test = df_test.copy()
train['is_test'] = False
test['is_test'] = True
df_whole = pd.concat([train, test])

In [12]:
train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

# Name の一致度から Publisher を探す

In [14]:
# もう nan はないのを確認
df_test[df_test.loc[:, 'Publisher'].isna()]

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales


In [20]:
# 名前が nan は2件あるので処理しておく
display(df_test[df_test.loc[:, 'Name'].isna()])
df_test['Name'] = df_test['Name'].fillna('NaN')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
278,NaN,GEN,1993.0,NaN,Acclaim Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,8637,48.062438
279,NaN,GEN,1993.0,NaN,Acclaim Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,8638,23.178798


In [23]:
# unknown はあと165個
df_test_unk = df_test[df_test.loc[:, 'Publisher'] == 'Unknown']
df_test_unk.shape

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(165, 13)

In [22]:
# Maddeon NFL を見ると Electronic Arts とわかる。test でかぶってるやつもあるよね
df_test[df_test.loc[:, 'Name'].str.contains('Madden NFL')]

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
145,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,94.0,29.0,8.5,140.0,EA Tiburon,E,8504,114.953023
148,Madden NFL 07,PSP,NaN,Sports,Unknown,78.0,18.0,6.6,16.0,EA Tiburon,E,8507,39.277323
152,Madden NFL 11,Wii,NaN,Sports,Unknown,75.0,6.0,5.4,7.0,EA Tiburon,E,8511,28.425216
153,Madden NFL 06,X360,NaN,Sports,Electronic Arts,74.0,48.0,4.9,44.0,EA Tiburon,E,8512,92.190305
154,Madden NFL 2002,XB,NaN,Sports,Unknown,90.0,16.0,8.1,8.0,EA Sports,E,8513,25.059178
546,Madden NFL 97,PS,1996.0,Sports,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,8905,80.530725
699,Madden NFL 98,PS,1997.0,Sports,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,9058,80.763161
883,Madden NFL 2000,PS,1998.0,Sports,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,9242,90.062777
884,Madden NFL 99,PS,1998.0,Sports,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,9243,93.108214
892,Madden NFL 99,N64,1998.0,Sports,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,9251,27.854975


In [27]:
df_test_known = df_test.drop(df_test_unk.index, axis=0)
df_test_known.shape

(8195, 13)

In [34]:
count = 0
count_series = 0
count_trigram = 0
count_bigram = 0
count_unigram = 0

for i, name in zip(df_test_unk.index, df_test_unk['Name']):
    df_tmp = pd.DataFrame()
    if ':' in name:
        name = name.split(':')[0]
        df_tmp = df_test_known[df_test_known['Name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_series += 1
    if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 3:
        name = ' '.join(name.split(' ')[:3])
        df_tmp = df_test_known[df_test_known['Name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_trigram += 1
    if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 2:
        name = ' '.join(name.split(' ')[:2])
        df_tmp = df_test_known[df_test_known['Name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_bigram += 1
#     if df_tmp.shape[0] == 0:
#         name = name.split(' ')[0]
#         if name != 'The':
#             df_tmp = df_train[df_train['Name'].str.startswith(name)]
#             count_unigram += 1

    if df_tmp.shape[0] > 0:
        print(name)
        display(df_tmp)
        count += 1
        most_publisher = df_tmp.loc[:, 'Publisher'].value_counts().reset_index().loc[0, 'index']
        if df_test_unk.loc[i, 'Publisher'] != 'Unknown':
            raise
        df_test_unk.loc[i, 'Publisher'] = most_publisher
print(count, count_series, count_trigram, count_bigram, count_unigram)

WCW Backstage Assault


/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
965,WCW Backstage Assault,PS,1999.0,Action,Electronic Arts,40.0,9.0,NaN,NaN,Kodiak Interactive,T,9324,18.693782


Rock Revolution


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
4523,Rock Revolution,X360,2008.0,Misc,Konami Digital Entertainment,38.0,28.0,5.1,17.0,"Zoe Mode, HB Studios Multimedia",T,12882,45.658812
4549,Rock Revolution,DS,2008.0,Misc,Konami Digital Entertainment,NaN,NaN,6.8,8.0,"Zoe Mode, HB Studios Multimedia",E10+,12908,48.470551


...

Horse Life


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
4159,Horse Life,DS,2007.0,Simulation,Game Life,68.0,9.0,tbd,NaN,Neko Entertainment,E,12518,8.800543
4820,Horse Life Adventures,Wii,2008.0,Simulation,Deep Silver,NaN,NaN,tbd,NaN,Neko Entertainment,E,13179,21.910418
4826,Horse Life Adventures,DS,2008.0,Simulation,Deep Silver,NaN,NaN,tbd,NaN,Neko Entertainment,E,13185,25.204486


Ao no


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
7021,Ao no Exorcist: Genkoku no Labyrinth,PSP,2012.0,Action,Namco Bandai Games,NaN,NaN,NaN,NaN,NaN,NaN,15380,4.171526


45 9 18 18 0


In [36]:
df_test_unk.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,Global_Sales
5,Cubix Robots for Everyone: Clash 'n' Bash,GBA,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8364,5.408323
17,WCW Backstage Assault,N64,NaN,Action,Electronic Arts,NaN,NaN,NaN,NaN,NaN,NaN,8376,14.427532
19,Action Man-Operation Extreme,PS,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8378,4.685558
23,Housekeeping,DS,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8382,3.495931
25,Super Duper Sumos,GBA,NaN,Action,Unknown,57.0,5.0,tbd,NaN,Handheld Games,E,8384,6.827206


In [ ]:
# 90個を整理

In [39]:
df_final_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'id_test_unknown1.csv', index=False)

165
120


In [42]:
df_final_test = pd.read_csv(datadir / 'country_prob_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'country_prob_test_unknown1.csv', index=False)

165
120


# train と test で単語的に被ってるやつを消した。更に減らす

In [47]:
df_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
df_test_unk = df_test[df_test['Publisher'] == 'Unknown']
df_test_known = df_test.drop(df_test_unk.index, axis=0)
df_known = pd.concat([df_train, df_test_known], axis=0)
df_known['Name'] = df_known['Name'].fillna('NaN')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
df_test_unk

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id
5,Cubix Robots for Everyone: Clash 'n' Bash,GBA,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8364
19,Action Man-Operation Extreme,PS,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8378
23,Housekeeping,DS,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8382
25,Super Duper Sumos,GBA,NaN,Action,Unknown,57.0,5.0,tbd,NaN,Handheld Games,E,8384
30,The Daring Game for Girls,DS,NaN,Adventure,Unknown,NaN,NaN,tbd,NaN,WXP,E,8389
31,The Daring Game for Girls,Wii,NaN,Adventure,Unknown,NaN,NaN,tbd,NaN,WXP,E,8390
32,The Hidden,3DS,NaN,Adventure,Unknown,NaN,NaN,4.2,5.0,1st Playable Productions,E10+,8391
33,B.L.U.E.: Legend of Water,PS,NaN,Adventure,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8392
36,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8395
42,Swords,Wii,NaN,Fighting,Unknown,NaN,NaN,tbd,NaN,Panic Button,T,8401


In [65]:
df_known[df_known['Name'].str.contains('Raw')]

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Critic_Count,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales
1190,Destruction Derby Raw,PS,2000.0,Racing,Sony Computer Entertainment,8.0,5.0,0.0,1.0,14.0,...,9.0,7.3,14.0,Studio 33,E,1190,True,True,False,True
1570,WWE SmackDown! vs. Raw,PS2,2002.0,Fighting,THQ,132.0,108.0,4.0,39.0,283.0,...,NaN,NaN,NaN,NaN,NaN,1570,True,True,True,True
1575,WWF Raw,XB,2002.0,Fighting,THQ,40.0,13.0,0.0,2.0,55.0,...,33.0,6.6,15.0,Anchor,T,1575,True,True,False,True
1952,WWE Raw 2,XB,2003.0,Fighting,THQ,43.0,11.0,0.0,2.0,56.0,...,26.0,7.3,25.0,Anchor,T,1952,True,True,False,True
3214,Raw Danger! (JP sales),PS2,2006.0,Action,505 Games,0.0,0.0,6.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,3214,False,False,True,False
3230,Raw Danger!,PS2,2006.0,Action,505 Games,2.0,1.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,3230,True,True,False,False
3813,WWE SmackDown vs Raw 2008,PS2,2007.0,Fighting,THQ,92.0,0.0,1.0,141.0,234.0,...,11.0,7.2,31.0,Yuke's,T,3813,True,False,True,True
3814,WWE SmackDown vs Raw 2008,X360,2007.0,Fighting,THQ,92.0,38.0,0.0,13.0,143.0,...,41.0,6.8,48.0,Yuke's,T,3814,True,True,False,True
3815,WWE SmackDown vs Raw 2008,PS3,2007.0,Fighting,THQ,62.0,49.0,1.0,20.0,133.0,...,27.0,7.3,41.0,Yuke's,T,3815,True,True,True,True
3816,WWE SmackDown vs Raw 2008,PSP,2007.0,Fighting,THQ,45.0,47.0,0.0,28.0,119.0,...,11.0,7.8,15.0,Yuke's,T,3816,True,True,False,True


## stopword 消して lower にしたら増えそう -> 4件発見

In [67]:
import texthero
from texthero import preprocessing

def _clean(series: pd.Series) -> pd.Series:
    custom_pipeline = [
        preprocessing.fillna,
        preprocessing.lowercase,
        preprocessing.remove_digits,
        preprocessing.remove_punctuation,
        preprocessing.remove_diacritics,
        preprocessing.remove_whitespace
    ]
    return texthero.clean(series, pipeline=custom_pipeline)

df_known['clean_name'] = _clean(df_known['Name'])
df_test_unk['clean_name'] = _clean(df_test_unk['Name'])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [69]:
count = 0
count_series = 0
count_trigram = 0
count_bigram = 0
count_unigram = 0

for i, name in zip(df_test_unk.index, df_test_unk['clean_name']):
    df_tmp = pd.DataFrame()
    if ':' in name:
        name = name.split(':')[0]
        df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_series += 1
    if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 3:
        name = ' '.join(name.split(' ')[:3])
        df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_trigram += 1
    if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 2:
        name = ' '.join(name.split(' ')[:2])
        df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
        if df_tmp.shape[0] != 0:
            count_bigram += 1
#     if df_tmp.shape[0] == 0:
#         name = name.split(' ')[0]
#         if name != 'The':
#             df_tmp = df_train[df_train['Name'].str.startswith(name)]
#             count_unigram += 1

    if df_tmp.shape[0] > 0:
        print(name)
        display(df_tmp)
        count += 1
        most_publisher = df_tmp.loc[:, 'Publisher'].value_counts().reset_index().loc[0, 'index']
        if df_test_unk.loc[i, 'Publisher'] != 'Unknown':
            raise
        df_test_unk.loc[i, 'Publisher'] = most_publisher
print(count, count_series, count_trigram, count_bigram, count_unigram)

wwe smackdown vs


/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name
1570,WWE SmackDown! vs. Raw,PS2,2002.0,Fighting,THQ,132.0,108.0,4.0,39.0,283.0,...,NaN,NaN,NaN,NaN,1570,True,True,True,True,wwe smackdown vs raw
2801,WWE SmackDown! vs. RAW 2006,PS2,2005.0,Fighting,THQ,145.0,111.0,4.0,33.0,294.0,...,NaN,NaN,NaN,NaN,2801,True,True,True,True,wwe smackdown vs raw
2809,WWE SmackDown! vs. RAW 2006,PSP,2005.0,Fighting,THQ,36.0,0.0,0.0,3.0,40.0,...,8.5,37.0,Yuke's,T,2809,True,False,False,True,wwe smackdown vs raw
3297,WWE SmackDown vs. RAW 2007,PS2,2006.0,Fighting,THQ,140.0,88.0,3.0,26.0,258.0,...,8.6,57.0,Yuke's,T,3297,True,True,True,True,wwe smackdown vs raw
3301,WWE SmackDown vs. RAW 2007,PSP,2006.0,Fighting,THQ,33.0,20.0,0.0,14.0,66.0,...,8.1,21.0,Yuke's,T,3301,True,True,False,True,wwe smackdown vs raw
3303,WWE SmackDown vs. RAW 2007,X360,2006.0,Fighting,THQ,44.0,3.0,0.0,4.0,50.0,...,7.8,46.0,Yuke's,T,3303,True,True,False,True,wwe smackdown vs raw
3813,WWE SmackDown vs Raw 2008,PS2,2007.0,Fighting,THQ,92.0,0.0,1.0,141.0,234.0,...,7.2,31.0,Yuke's,T,3813,True,False,True,True,wwe smackdown vs raw
3814,WWE SmackDown vs Raw 2008,X360,2007.0,Fighting,THQ,92.0,38.0,0.0,13.0,143.0,...,6.8,48.0,Yuke's,T,3814,True,True,False,True,wwe smackdown vs raw
3815,WWE SmackDown vs Raw 2008,PS3,2007.0,Fighting,THQ,62.0,49.0,1.0,20.0,133.0,...,7.3,41.0,Yuke's,T,3815,True,True,True,True,wwe smackdown vs raw
3816,WWE SmackDown vs Raw 2008,PSP,2007.0,Fighting,THQ,45.0,47.0,0.0,28.0,119.0,...,7.8,15.0,Yuke's,T,3816,True,True,False,True,wwe smackdown vs raw


beyond the


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name
477,Beyond The Beyond,PS,1995.0,Role-Playing,Sony Computer Entertainment,11.0,8.0,35.0,4.0,56.0,...,NaN,NaN,NaN,NaN,477,True,True,True,True,beyond the beyond


m m s


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name
4010,M&M's Kart Racing,Wii,2007.0,Racing,Zoo Digital Publishing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12369,NaN,NaN,NaN,NaN,m m s kart racing
4431,M&M's Adventure,DS,2008.0,Adventure,Zoo Digital Publishing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12790,NaN,NaN,NaN,NaN,m m s adventure
4671,M&M's Kart Racing,DS,2008.0,Racing,Zoo Digital Publishing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13030,NaN,NaN,NaN,NaN,m m s kart racing
5320,M&M's Beach Party,Wii,2009.0,Misc,Zoo Digital Publishing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,13679,NaN,NaN,NaN,NaN,m m s beach party


chibi robo


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name
2777,Chibi-Robo! Plug into Adventure!,GC,2005.0,Adventure,Nintendo,23.0,6.0,9.0,1.0,39.0,...,NaN,NaN,NaN,NaN,2777,True,True,True,True,chibi robo plug into adventure
7983,Chibi-Robo! Zip Lash,3DS,2015.0,Platform,Nintendo,9.0,8.0,6.0,2.0,24.0,...,NaN,NaN,NaN,NaN,7983,True,True,True,True,chibi robo zip lash


4 0 2 2 0


In [71]:
df_test_unk.shape, df_test_unk[df_test_unk['Publisher'] == 'Unknown'].shape

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((120, 13), (116, 13))

In [72]:
df_final_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'id_test_unknown1.csv', index=False)

120
116


In [73]:
df_final_test = pd.read_csv(datadir / 'country_prob_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'country_prob_test_unknown1.csv', index=False)

120
116


# 次, 1文字目見ちゃうか?

In [137]:
df_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
df_test_unk = df_test[df_test['Publisher'] == 'Unknown']
df_test_known = df_test.drop(df_test_unk.index, axis=0)
df_known = pd.concat([df_train, df_test_known], axis=0)
df_known['Name'] = df_known['Name'].fillna('NaN')

In [138]:
df_known['clean_name'] = _clean(df_known['Name'])
df_test_unk['clean_name'] = _clean(df_test_unk['Name'])

In [139]:
df_test_unk[df_test_unk['Name'].str.contains(':')]

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,clean_name
5,Cubix Robots for Everyone: Clash 'n' Bash,GBA,NaN,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8364,cubix robots for everyone clash n bash
33,B.L.U.E.: Legend of Water,PS,NaN,Adventure,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,8392,b l u e legend of water
122,Tribes: Aerial Assault,PS2,NaN,Shooter,Unknown,73.0,23.0,8.8,47.0,Midway Studios - Austin,T,8481,tribes aerial assault
126,Combat Elite: WWII Paratroopers,PS2,NaN,Shooter,Unknown,54.0,12.0,tbd,NaN,BattleBorne,T,8485,combat elite wwii paratroopers
128,Combat Elite: WWII Paratroopers,XB,NaN,Shooter,Unknown,56.0,11.0,tbd,NaN,BattleBorne,T,8487,combat elite wwii paratroopers
2494,Nicktoons Collection: Game Boy Advance Video V...,GBA,2004.0,Misc,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,10853,nicktoons collection game boy advance video vo...
2505,Sonic X: Game Boy Advance Video Volume 1,GBA,2004.0,Misc,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,10864,sonic x game boy advance video volume
2511,Nicktoons Collection: Game Boy Advance Video V...,GBA,2004.0,Misc,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,10870,nicktoons collection game boy advance video vo...
2939,Nicktoons Collection: Game Boy Advance Video V...,GBA,2005.0,Misc,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,11298,nicktoons collection game boy advance video vo...
4297,UFO: Trilogy,PC,2007.0,Strategy,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,12656,ufo trilogy


In [140]:
df_known[df_known['clean_name'].str.contains('Thomas'.lower())]

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name
7893,Thomas and Friends: Steaming around Sodor,3DS,2015.0,Action,Avanquest Software,0.0,2.0,0.0,0.0,2.0,...,tbd,NaN,Avanquest Software,NaN,7893,False,True,False,False,thomas and friends steaming around sodor
564,Frank Thomas Big Hurt Baseball,PS,1996.0,Sports,Acclaim Entertainment,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8923,NaN,NaN,NaN,NaN,frank thomas big hurt baseball


In [141]:
def colon_latter(x: str):
    if ':' in x:
        return x.split(':')[1].lower().strip()
    return ''

def colon_former(x: str):
    if ':' in x:
        return x.split(':')[0].lower().strip()
    return ''
df_known['Name_colon_latter'] = df_known['Name'].apply(colon_latter)
df_known['Name_colon_former'] = df_known['Name'].apply(colon_former)
df_known['Name_colon_former_lt_2'] = df_known['Name_colon_former'].apply(lambda x: len(x.split(' ')) <= 2)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
count = 0
count_series = 0
count_trigram = 0
count_bigram = 0
count_unigram = 0

for i, name, original in zip(df_test_unk.index, df_test_unk['clean_name'], df_test_unk['Name']):
    df_tmp = pd.DataFrame()
    if ':' in original:
        name_latter = original.split(':')[1].strip()
        name_latter = ' '.join(name_latter.lower().split(' ')[:2])
        df_tmp = df_known[df_known['Name_colon_latter'].str.startswith(name_latter)]
        if df_tmp.shape[0] != 0:
            count_series += 1
#     if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 3:
#         name = ' '.join(name.split(' ')[:3])
#         df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
#         if df_tmp.shape[0] != 0:
#             count_trigram += 1
#     if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 2:
#         name = ' '.join(name.split(' ')[:2])
#         df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
#         if df_tmp.shape[0] != 0:
#             count_bigram += 1
#     if df_tmp.shape[0] == 0:
#         name = name.split(' ')[0]
#         if name != 'The':
#             df_tmp = df_train[df_train['Name'].str.startswith(name)]
#             count_unigram += 1

    if df_tmp.shape[0] > 0:
        print(name)
        display(df_tmp)
        count += 1
#         most_publisher = df_tmp.loc[:, 'Publisher'].value_counts().reset_index().loc[0, 'index']
#         if df_test_unk.loc[i, 'Publisher'] != 'Unknown':
#             raise
#         df_test_unk.loc[i, 'Publisher'] = most_publisher
print(count, count_series, count_trigram, count_bigram, count_unigram)

b l u e legend of water


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name,Name_colon_latter
595,Super Mario RPG: Legend of the Seven Stars,SNES,1996.0,Role-Playing,Nintendo,66.0,0.0,145.0,3.0,214.0,...,NaN,NaN,NaN,595,True,False,True,True,super mario rpg legend of the seven stars,legend of the seven stars
3320,Kengo: Legend of The 9,X360,2006.0,Fighting,Majesco Entertainment,6.0,1.0,0.0,1.0,7.0,...,14.0,Genki,M,3320,True,True,False,True,kengo legend of the,legend of the 9
6682,Jewel Link Chronicles: Legend of Athena,DS,2011.0,Puzzle,GSP,0.0,6.0,0.0,1.0,7.0,...,NaN,NaN,NaN,6682,False,True,False,True,jewel link chronicles legend of athena,legend of athena
628,Croc: Legend of the Gobbos,PS,1997.0,Platform,Fox Interactive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8987,NaN,NaN,NaN,NaN,croc legend of the gobbos,legend of the gobbos
1354,Jackie Chan Adventures: Legend of the Dark Hand,GBA,2001.0,Action,Activision,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9713,NaN,NaN,NaN,NaN,jackie chan adventures legend of the dark hand,legend of the dark hand
3338,Pirates: Legend of the Black Buccaneer,PS2,2006.0,Adventure,10TACLE Studios,NaN,NaN,NaN,NaN,NaN,...,5.0,WideScreen Games,T,11697,NaN,NaN,NaN,NaN,pirates legend of the black buccaneer,legend of the black buccaneer
7888,Gravity Falls: Legend of the Gnome Gemulets,3DS,2015.0,Action,Ubisoft,NaN,NaN,NaN,NaN,NaN,...,10.0,Ubisoft Osaka,E,16247,NaN,NaN,NaN,NaN,gravity falls legend of the gnome gemulets,legend of the gnome gemulets


nicktoons collection game boy advance video volume


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name,Name_colon_latter
2495,SpongeBob SquarePants: Game Boy Advance Video ...,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10854,NaN,NaN,NaN,NaN,spongebob squarepants game boy advance video v...,game boy advance video volume 1
2497,SpongeBob SquarePants: Game Boy Advance Video ...,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10856,NaN,NaN,NaN,NaN,spongebob squarepants game boy advance video v...,game boy advance video volume 2
2500,The Fairly Odd Parents: Game Boy Advance Video...,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10859,NaN,NaN,NaN,NaN,the fairly odd parents game boy advance video ...,game boy advance video volume 1
2501,The Fairly Odd Parents: Game Boy Advance Video...,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10860,NaN,NaN,NaN,NaN,the fairly odd parents game boy advance video ...,game boy advance video volume 2
2506,Dora the Explorer: Game Boy Advance Video Volu...,GBA,2004.0,Misc,Take-Two Interactive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10865,NaN,NaN,NaN,NaN,dora the explorer game boy advance video volume,game boy advance video volume 1
2507,Cartoon Network Collection: Game Boy Advance V...,GBA,2004.0,Misc,Crave Entertainment,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10866,NaN,NaN,NaN,NaN,cartoon network collection game boy advance vi...,game boy advance video volume 1
2508,All Grown Up!: Game Boy Advance Video Volume 1,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10867,NaN,NaN,NaN,NaN,all grown up game boy advance video volume,game boy advance video volume 1
2512,SpongeBob SquarePants: Game Boy Advance Video ...,GBA,2004.0,Misc,THQ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10871,NaN,NaN,NaN,NaN,spongebob squarepants game boy advance video v...,game boy advance video volume 3
2515,Dragon Ball GT: Game Boy Advance Video Volume 1,GBA,2004.0,Misc,Atari,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10874,NaN,NaN,NaN,NaN,dragon ball gt game boy advance video volume,game boy advance video volume 1
2518,Teenage Mutant Ninja Turtles: Game Boy Advance...,GBA,2004.0,Misc,Palcom,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10877,NaN,NaN,NaN,NaN,teenage mutant ninja turtles game boy advance ...,game boy advance video volume 1


sonic x game boy advance video volume


...

black rock shooter the game


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name,Name_colon_latter
3253,24: The Game,PS2,2006.0,Adventure,Sony Computer Entertainment,15.0,12.0,0.0,4.0,30.0,...,40.0,SCE Studio Cambridge,M,3253,True,True,False,True,the game,the game
3983,Napoleon Dynamite: The Game,PSP,2007.0,Puzzle,Crave Entertainment,3.0,0.0,0.0,0.0,4.0,...,7.0,7 Studios,E10+,3983,True,False,False,False,napoleon dynamite the game,the game
3986,Napoleon Dynamite: The Game,DS,2007.0,Puzzle,Crave Entertainment,2.0,0.0,0.0,0.0,3.0,...,4.0,7 Studios,E10+,3986,True,False,False,False,napoleon dynamite the game,the game
4463,Igor: The Game,DS,2008.0,Adventure,Legacy Interactive,4.0,0.0,0.0,0.0,4.0,...,NaN,Artefacts Studio,E,4463,True,False,False,False,igor the game,the game
4552,Peppa Pig: The Game,DS,2008.0,Misc,Pinnacle,0.0,81.0,0.0,6.0,87.0,...,NaN,NaN,NaN,4552,False,True,False,True,peppa pig the game,the game
5244,Wipeout: The Game,Wii,2009.0,Misc,Mindscape,194.0,0.0,0.0,12.0,206.0,...,NaN,NaN,NaN,5244,True,False,False,True,wipeout the game,the game
5269,Peppa Pig: The Game,Wii,2009.0,Misc,Pinnacle,0.0,34.0,0.0,3.0,37.0,...,NaN,NaN,NaN,5269,False,True,False,True,peppa pig the game,the game
5393,Countdown: The Game,DS,2009.0,Puzzle,Mindscape,0.0,14.0,0.0,1.0,15.0,...,NaN,NaN,NaN,5393,False,True,False,True,countdown the game,the game
5406,Countdown: The Game,Wii,2009.0,Puzzle,Mindscape,0.0,5.0,0.0,0.0,5.0,...,NaN,NaN,NaN,5406,False,True,False,False,countdown the game,the game
6061,Despicable Me: The Game,Wii,2010.0,Platform,D3Publisher,16.0,8.0,0.0,2.0,27.0,...,NaN,NaN,NaN,6061,True,True,False,True,despicable me the game,the game


demolition company gold edition


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales,clean_name,Name_colon_latter
4276,Age of Empires III: Gold Edition,PC,2007.0,Strategy,Microsoft Game Studios,0.0,2.0,0.0,0.0,3.0,...,6.0,Ensemble Studios,T,4276,False,True,False,False,age of empires iii gold edition,gold edition
6226,Serious Sam HD: Gold Edition,PC,2010.0,Shooter,Devolver Digital,0.0,2.0,0.0,0.0,2.0,...,NaN,NaN,NaN,6226,False,True,False,False,serious sam hd gold edition,gold edition
6355,Tropico 3: Gold Edition,PC,2010.0,Strategy,Kalypso Media,0.0,1.0,0.0,0.0,1.0,...,NaN,Haemimont,T,6355,False,True,False,False,tropico gold edition,gold edition
6816,Serious Sam HD: Gold Edition,X360,2011.0,Shooter,Mastertronic,0.0,2.0,0.0,0.0,2.0,...,NaN,NaN,NaN,6816,False,True,False,False,serious sam hd gold edition,gold edition
1489,Soldier of Fortune: Gold Edition,PS2,2001.0,Shooter,Codemasters,NaN,NaN,NaN,NaN,NaN,...,17.0,Pipe Dream Interactive,M,9848,NaN,NaN,NaN,NaN,soldier of fortune gold edition,gold edition
2052,RollerCoaster Tycoon: Gold Edition,PC,2002.0,Strategy,Infogrames,NaN,NaN,NaN,NaN,NaN,...,NaN,Infogrames,E,10411,NaN,NaN,NaN,NaN,rollercoaster tycoon gold edition,gold edition


14 14 0 0 0


In [142]:
count = 0
count_series = 0
count_trigram = 0
count_bigram = 0
count_unigram = 0

for i, name, original in zip(df_test_unk.index, df_test_unk['clean_name'], df_test_unk['Name']):
    df_tmp = pd.DataFrame()
    if ':' in original:
        name_former = original.split(':')[0]
        if len(name_former.lower().split(' ')) <= 2:
            name_former = name_former.lower().split(' ')[0]
            df_tmp = df_known[df_known['Name_colon_former_lt_2'] & df_known['Name'].str.contains(':') & df_known['clean_name'].str.startswith(name_former + ' ')]
        if df_tmp.shape[0] != 0:
            count_series += 1
#     if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 3:
#         name = ' '.join(name.split(' ')[:3])
#         df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
#         if df_tmp.shape[0] != 0:
#             count_trigram += 1
#     if df_tmp.shape[0] == 0 and len(name.split(' ')) >= 2:
#         name = ' '.join(name.split(' ')[:2])
#         df_tmp = df_known[df_known['clean_name'].str.startswith(name)]
#         if df_tmp.shape[0] != 0:
#             count_bigram += 1
#     if df_tmp.shape[0] == 0:
#         name = name.split(' ')[0]
#         if name != 'The':
#             df_tmp = df_train[df_train['Name'].str.startswith(name)]
#             count_unigram += 1

    if df_tmp.shape[0] > 0:
        print(name)
#         display(df_tmp)
        count += 1
        most_publisher = df_tmp.loc[:, 'Publisher'].value_counts().reset_index().loc[0]
        if most_publisher['Publisher'] > 1:
            if df_test_unk.loc[i, 'Publisher'] != 'Unknown':
                raise
            df_test_unk.loc[i, 'Publisher'] = most_publisher['index']
print(count, count_series, count_trigram, count_bigram, count_unigram)

combat elite wwii paratroopers
combat elite wwii paratroopers
nicktoons collection game boy advance video volume
Unknown
sonic x game boy advance video volume
Unknown
nicktoons collection game boy advance video volume
Unknown
nicktoons collection game boy advance video volume
Unknown
kid adventures sky captain
power gig rise of the sixstring
Unknown
power gig rise of the sixstring
Unknown
vampire mansion linda hyde
dark parables the exiled prince
real crimes the unicorn killer
sengoku otome legend battle
Unknown
13 13 0 0 0


In [143]:
df_test_unk.shape, df_test_unk[df_test_unk['Publisher'] == 'Unknown'].shape

((116, 13), (109, 13))

In [146]:
df_final_test = pd.read_csv(datadir / 'id_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'id_test_unknown.csv', index=False)

109
109


In [147]:
df_final_test = pd.read_csv(datadir / 'country_prob_test_unknown1.csv')
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.loc[df_test_unk.index, 'Publisher'] = df_test_unk.loc[:, 'Publisher']
print(df_final_test[df_final_test['Publisher'] == 'Unknown'].shape[0])
df_final_test.to_csv(datadir / 'country_prob_test_unknown.csv', index=False)

109
109
